<a href="https://colab.research.google.com/github/lankipolo123/roadfixqc/blob/main/pothole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# YOLOv11 Pothole Detection Training - Ultralytics
# Optimized for 90% Accuracy

# ============================================================================
# STEP 1: INSTALL AND SETUP
# ============================================================================

!pip install ultralytics
!pip install roboflow

import os
from ultralytics import YOLO
import torch

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================================
# STEP 2: DOWNLOAD DATASET FROM ROBOFLOW
# ============================================================================

from roboflow import Roboflow
rf = Roboflow(api_key="SzttdelfmuWaCwAz2N5u")
project = rf.workspace("dequillaprojects").project("pothole-ol3a7")
version = project.version(4)
dataset = version.download("yolov11")

# Get dataset info
dataset_path = dataset.location
print(f"Dataset downloaded to: {dataset_path}")

# Count images
train_images = len(os.listdir(f"{dataset_path}/train/images"))
val_images = len(os.listdir(f"{dataset_path}/valid/images"))
print(f"Training images: {train_images}")
print(f"Validation images: {val_images}")

# ============================================================================
# STEP 3: TRAIN YOLOV11N (NANO) - OPTIMIZED
# ============================================================================

print("Training YOLOv11n (Nano)...")

# Load YOLOv11n model
model_n = YOLO('yolo11n.pt')

# Train with optimized settings for 90% accuracy
results_n = model_n.train(
    data=f'{dataset_path}/data.yaml',
    epochs=200,
    imgsz=832,          # Higher resolution for better detection
    batch=24,           # Nano can handle larger batches
    lr0=0.001,          # Learning rate
    lrf=0.01,           # Final learning rate factor
    optimizer='AdamW',  # Better optimizer
    cos_lr=True,        # Cosine learning rate scheduler
    patience=50,        # Early stopping
    save_period=10,     # Save every 10 epochs

    # Augmentation optimized for potholes
    hsv_h=0.01,         # Slight hue changes
    hsv_s=0.5,          # Saturation changes
    hsv_v=0.3,          # Brightness changes
    degrees=15,         # Rotation up to 15 degrees
    translate=0.1,      # Translation
    scale=0.2,          # Scale variation
    shear=10,           # Shear transformation
    fliplr=0.5,         # 50% horizontal flip
    flipud=0.0,         # No vertical flip for roads
    mosaic=1.0,         # Mosaic augmentation
    mixup=0.1,          # Mixup augmentation

    # Loss weights
    box=7.5,            # Box loss weight
    cls=0.5,            # Classification loss
    dfl=1.5,            # DFL loss

    name='yolov11n_pothole',
    project='pothole_training',
    exist_ok=True
)

# Validate YOLOv11n
val_results_n = model_n.val()
print(f"YOLOv11n Results:")
print(f"mAP50: {val_results_n.box.map50:.4f}")
print(f"mAP50-95: {val_results_n.box.map:.4f}")

# ============================================================================
# STEP 4: TRAIN YOLOV11S (SMALL) - OPTIMIZED
# ============================================================================

print("Training YOLOv11s (Small)...")

# Load YOLOv11s model
model_s = YOLO('yolo11s.pt')

# Train with optimized settings
results_s = model_s.train(
    data=f'{dataset_path}/data.yaml',
    epochs=200,
    imgsz=832,          # Higher resolution
    batch=16,           # Smaller batch for larger model
    lr0=0.001,          # Learning rate
    lrf=0.01,           # Final learning rate factor
    optimizer='AdamW',  # Better optimizer
    cos_lr=True,        # Cosine learning rate scheduler
    patience=50,        # Early stopping
    save_period=10,     # Save every 10 epochs

    # Augmentation optimized for potholes
    hsv_h=0.01,         # Slight hue changes
    hsv_s=0.5,          # Saturation changes
    hsv_v=0.3,          # Brightness changes
    degrees=15,         # Rotation up to 15 degrees
    translate=0.1,      # Translation
    scale=0.2,          # Scale variation
    shear=10,           # Shear transformation
    fliplr=0.5,         # 50% horizontal flip
    flipud=0.0,         # No vertical flip for roads
    mosaic=1.0,         # Mosaic augmentation
    mixup=0.1,          # Mixup augmentation

    # Loss weights
    box=7.5,            # Box loss weight
    cls=0.5,            # Classification loss
    dfl=1.5,            # DFL loss

    name='yolov11s_pothole',
    project='pothole_training',
    exist_ok=True
)

# Validate YOLOv11s
val_results_s = model_s.val()
print(f"YOLOv11s Results:")
print(f"mAP50: {val_results_s.box.map50:.4f}")
print(f"mAP50-95: {val_results_s.box.map:.4f}")

# ============================================================================
# STEP 5: COMPARE RESULTS AND SELECT BEST MODEL
# ============================================================================

print("\n" + "="*50)
print("RESULTS COMPARISON")
print("="*50)
print(f"YOLOv11n mAP50: {val_results_n.box.map50:.4f}")
print(f"YOLOv11s mAP50: {val_results_s.box.map50:.4f}")

# Select best model
if val_results_s.box.map50 > val_results_n.box.map50:
    best_model = model_s
    best_results = val_results_s
    best_name = "YOLOv11s"
    best_path = "pothole_training/yolov11s_pothole/weights/best.pt"
else:
    best_model = model_n
    best_results = val_results_n
    best_name = "YOLOv11n"
    best_path = "pothole_training/yolov11n_pothole/weights/best.pt"

print(f"\nBest Model: {best_name}")
print(f"Best mAP50: {best_results.box.map50:.4f}")

# ============================================================================
# STEP 6: FINE-TUNING IF NEEDED (FOR 90% TARGET)
# ============================================================================

if best_results.box.map50 < 0.90:
    print(f"\nCurrent accuracy: {best_results.box.map50*100:.1f}%")
    print("Fine-tuning for 90% target...")

    # Load best model for fine-tuning
    fine_tune_model = YOLO(best_path)

    # Fine-tune with higher resolution and lower learning rate
    fine_tune_results = fine_tune_model.train(
        data=f'{dataset_path}/data.yaml',
        epochs=100,         # Fewer epochs for fine-tuning
        imgsz=1024,         # Even higher resolution
        batch=8,            # Smaller batch for higher resolution
        lr0=0.0001,         # Much lower learning rate
        optimizer='AdamW',
        cos_lr=True,
        patience=30,
        mosaic=0.5,         # Reduce mosaic for fine-tuning
        mixup=0.05,         # Reduce mixup
        name=f'{best_name.lower()}_finetune',
        project='pothole_training',
        exist_ok=True
    )

    # Validate fine-tuned model
    final_results = fine_tune_model.val()
    print(f"\nFine-tuned Results:")
    print(f"Final mAP50: {final_results.box.map50:.4f}")

    # Update best model if fine-tuning improved results
    if final_results.box.map50 > best_results.box.map50:
        best_results = final_results
        best_path = f"pothole_training/{best_name.lower()}_finetune/weights/best.pt"

# ============================================================================
# STEP 7: FINAL EVALUATION AND TESTING
# ============================================================================

print("\n" + "="*50)
print("FINAL RESULTS")
print("="*50)

final_accuracy = best_results.box.map50 * 100
print(f"Final Accuracy (mAP50): {final_accuracy:.1f}%")
print(f"mAP50-95: {best_results.box.map:.4f}")
print(f"Precision: {best_results.box.mp:.4f}")
print(f"Recall: {best_results.box.mr:.4f}")

if final_accuracy >= 90:
    print("🎉 SUCCESS! Achieved 90%+ accuracy!")
else:
    print(f"📊 Current: {final_accuracy:.1f}% (Target: 90%)")
    print("\n💡 Tips to improve:")
    print("   - Add more training data")
    print("   - Check data quality and annotations")
    print("   - Try ensemble methods")
    print("   - Increase training epochs")

# Save final model with clear name
import shutil
final_model_name = f"best_pothole_model_{final_accuracy:.1f}percent.pt"
shutil.copy(best_path, final_model_name)
print(f"\nFinal model saved as: {final_model_name}")

# ============================================================================
# STEP 8: TEST PREDICTIONS ON SAMPLE IMAGES
# ============================================================================

print("\nTesting predictions...")

# Load the best model for inference
final_model = YOLO(best_path)

# Run predictions on validation images
val_image_dir = f"{dataset_path}/valid/images"
sample_images = os.listdir(val_image_dir)[:5]  # Test on 5 images

for img_name in sample_images:
    img_path = os.path.join(val_image_dir, img_name)
    results = final_model.predict(
        source=img_path,
        conf=0.25,      # Confidence threshold
        save=True,      # Save results
        show_labels=True,
        show_conf=True
    )
    print(f"Predicted on: {img_name}")

print("\nPrediction images saved in 'runs/detect/predict' folder")

# ============================================================================
# STEP 9: MODEL EXPORT (OPTIONAL)
# ============================================================================

print("\nExporting model for deployment...")

# Export to different formats
final_model.export(format='onnx')    # ONNX format
final_model.export(format='torchscript')  # TorchScript
print("Model exported to ONNX and TorchScript formats")

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("="*50)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 132.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with


Extracting Dataset Version Zip to pothole-4 in yolov11:: 100%|██████████| 39452/39452 [00:05<00:00, 6778.02it/s]


Dataset downloaded to: /content/pothole-4
Training images: 16046
Validation images: 1914
Training YOLOv11n (Nano)...
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/pothole-4/data.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.3, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11n_p

In [ ]:
from google.colab import drive
drive.mount('/content/drive')